In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import json
import os
import pymongo as pm
from dotenv import load_dotenv
import csv


In [4]:
USER       = os.getenv("MONGO_USER")
PASS       = os.getenv("MONGO_PASS")
HOST       = os.getenv("MONGO_HOST")
PORT       = os.getenv("MONGO_PORT")
AUTH_DB    = os.getenv("MONGO_AUTH_DB")
DB_NAME    = os.getenv("MONGO_DB")

load_dotenv()

uri = f"mongodb://{USER}:{PASS}@{HOST}:{PORT}/?authSource={AUTH_DB}"
clientDB   = pm.MongoClient(uri)
db         = clientDB[DB_NAME]

# usa o hostname como WORKER_ID por padrão
# Collections
tasks_coll = db[os.getenv("MONGO_COLLECTION_TASKS")]
data_coll  = db[os.getenv("MONGO_COLLECTION_DATA")]

In [6]:
cursor = data_coll.find({
    "significant": True
})

In [15]:
posts = []
for doc in cursor:
    posts = doc.get('posts', [])
    break

In [16]:
posts

[{'text': 'Obrigado <3 <3',
  'created_at': '2025-05-25T02:54:00.793Z',
  'lang': 'pt'},
 {'text': 'Fiz mais uma endoscopia, vamos ver o que caraio ainda tenho no estômago.',
  'created_at': '2025-05-24T16:36:43.824Z',
  'lang': 'pt'},
 {'text': 'Alguém tem essas 4 cartas que estão faltando?? 🥺🥺🥺🥺🥺🥺',
  'created_at': '2025-05-22T16:22:42.313Z',
  'lang': 'pt'},
 {'text': 'Tão precioso *-*',
  'created_at': '2025-05-22T04:51:01.490Z',
  'lang': 'pt'},
 {'text': 'Profissão ruim da porra! Odeio o fato de que o que "quero" fazer envolve coisas que tenho extrema dificuldade de aprender. Tô a ponto de cuspir minha alma com um grito. Trabalhar com criança me adoece demais, mds, a vontade é de bater minha cabeça na parede até perder a consciência.',
  'created_at': '2025-05-21T17:47:44.658Z',
  'lang': 'pt'},
 {'text': "Mds, eu vou enlouquecer com essas crianças '-' eu ODEIO trabalhar no fundamental 1.",
  'created_at': '2025-05-21T12:04:32.424Z',
  'lang': 'pt'},
 {'text': 'O motivo de minha 

In [17]:
TARGET_TOKENS_PER_FILE = 4000
CHARS_PER_TOKEN_ESTIMATE = 2
MAX_JSON_STRING_LENGTH = int(TARGET_TOKENS_PER_FILE * CHARS_PER_TOKEN_ESTIMATE)

def split_tweet_texts_into_chunks(all_tweets_data):
    """
    Splits a list of tweet objects (all_tweets_data) into multiple smaller chunks
    kept in memory. Each chunk will contain only the text content (strings) of the tweets,
    extracted using the specified text_key.
    The size of each chunk (as a JSON serialized list of strings) is kept
    below MAX_JSON_STRING_LENGTH.
    Returns a list of chunks, where each chunk is a list of tweet text strings.
    """
    if not isinstance(all_tweets_data, list):
        return [] 

    if not all_tweets_data:
        return [] 

    all_text_chunks = []
    num_total_tweets = len(all_tweets_data)
    current_pos = 0

    while current_pos < num_total_tweets:
        current_chunk_texts = [] 
        
        for i in range(current_pos, num_total_tweets):
            tweet_dict = all_tweets_data[i] 
            
            if not isinstance(tweet_dict, dict):
                continue

            tweet_text_to_add = tweet_dict.get('text')

            if tweet_text_to_add is None:
                continue
            if not isinstance(tweet_text_to_add, str):
                continue


            # temp_chunk_for_testing now holds strings
            temp_chunk_for_testing = current_chunk_texts + [tweet_text_to_add]
            prospective_json_string = json.dumps(temp_chunk_for_testing, ensure_ascii=False, indent=None)
            prospective_len = len(prospective_json_string)

            if prospective_len <= MAX_JSON_STRING_LENGTH:
                current_chunk_texts.append(tweet_text_to_add)
            else:
                if not current_chunk_texts: # If the very first tweet text itself is too large
                    current_chunk_texts.append(tweet_text_to_add)
                break 
        
        num_texts_in_chunk = len(current_chunk_texts)

        if num_texts_in_chunk > 0:
            all_text_chunks.append(current_chunk_texts)
            current_pos += num_texts_in_chunk
        elif current_pos < num_total_tweets:
            if not current_chunk_texts and all_tweets_data[current_pos:]:
                 current_pos +=1 
            else:
                 break
    
    return all_text_chunks

if __name__ == "__main__":
    chunks = split_tweet_texts_into_chunks(all_tweets_data=posts)

In [18]:
TOTAL_TOKENS_ACCEPTED = 0.40e5
MAX_CHARS = TOTAL_TOKENS_ACCEPTED*CHARS_PER_TOKEN_ESTIMATE
curr_char = 0
final_list = []
for chunk in chunks:
    if curr_char < MAX_CHARS:
        curr_char += len("".join(chunk))
        final_list += chunk

len(final_list)

402

In [19]:
final_list


['Obrigado <3 <3',
 'Fiz mais uma endoscopia, vamos ver o que caraio ainda tenho no estômago.',
 'Alguém tem essas 4 cartas que estão faltando?? 🥺🥺🥺🥺🥺🥺',
 'Tão precioso *-*',
 'Profissão ruim da porra! Odeio o fato de que o que "quero" fazer envolve coisas que tenho extrema dificuldade de aprender. Tô a ponto de cuspir minha alma com um grito. Trabalhar com criança me adoece demais, mds, a vontade é de bater minha cabeça na parede até perder a consciência.',
 "Mds, eu vou enlouquecer com essas crianças '-' eu ODEIO trabalhar no fundamental 1.",
 'O motivo de minha tristesa esta noite é que quero um bolo de pote, mas está chovendo e não posso fazer o moço vir me entregar isso na chuva.',
 'Não tem água na escola. Quero ver como vou dar aula pra crianças até às 15h30m sem tomar água.',
 'Carai, quero trabalhar não.',
 'Eu menti? aushuahsuahsuhas',
 'Carinha pedindo...',
 "Nossa, isso aqui me quebra as pernas. Os correios não entregam em casa, vou ter que ir na cidade, dar sorte se tiver 

In [20]:
import datetime

if "__main__" == __name__:
    openai_api_key = os.getenv("OPENAI_API_KEY")
    import chatgpt as api_connection

    tweets = final_list
    response = api_connection.gerar_relatorio(openai_api_key,tweets=tweets)
    output_hour = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")
    print(response)

```json
{
    "Usuario": "INCERTO",
    "NomeProvavel": "INCERTO",
    "IdadeEstimada": "Adulto jovem",
    "GeneroEstimado": "INCERTO",
    "OrientacaoSexualSugestiva": "Gaysual",
    "RelacaoAfetivaSugerida": "Você curte o relacionamento do curandeiro e de si mesmo.",
    "ProfissaoOcupacao": "Professor",
    "EscolaridadeIndicada": "Possui formação universitária",
    "LocalizacaoProvavel": "INCERTO",
    "CidadesMencionadas": ["Recife", "Maceió", "Salvador", "Barueri", "Arapiraca"],
    "ReligiaoSugerida": "INCERTO",
    "PosicionamentoPolitico": "Esquerdista",
    "SaudeFisicaCitacoes": [
        "Gastrite",
        "H Pylori",
        "Problemas no estômago",
        "Necessidade de endoscopia",
        "Problemas nas costas"
    ],
    "SaudeMentalCitacoes": ["Ansiedade", "Depressão"],
    "UsoDeSubstancias": "Alprazolam mencionado",
    "TopicosRelevantes": [
        "Trabalho com crianças",
        "Educação",
        "Gastrite",
        "Justiça social"
    ],
    "HobbiesEIn